# Lecture 33 Notes

In these notes, we do an experiment to  the most efficient way to do linear in
Python. We will compare the performance of these functions:

In [18]:
def contains_in(x, lst):
    return x in lst

def contains_index(x, lst):
    """Return True if x is in lst, False otherwise. Uses the index method.
    If x is not found then index raises an exception. Exceptions are handled
    using a try-except block.
    """
    try:
        lst.index(x)
        return True
    except ValueError:
        return False

def contains_for_each(x, lst):
    for item in lst:
        if item == x:
            return True
    return False

def contains_for_range(x, lst):
    for i in range(len(lst)):
        if lst[i] == x:
            return True
    return False

def contains_while(x, lst):
    i = 0
    while i < len(lst):
        if lst[i] == x:
            return True
        i += 1
    return False

def contains_sentinel(x, lst):
    lst.append(x)
    i = 0
    while lst[i] != x:
        i += 1
    lst.pop()
    return i < len(lst)

`contains_sentinel(x, lst)` is an interesting implementation. It firsts appends
`x` to the end of `lst`: this `x` is the sentinel value. Then it iterates
through `lst` and makes `i` the index of the first occurrence of `x`. It then
removes the final `x` using `lst.pop()`. Finally, if `i` is less than the length
of `lst`, that means `x` is in `lst`, and so it returns `i`. Otherwise, the
looped stopped due to hitting the sentinel `x`, an so it returns `-1`.

## Recursive Linear Search

We're not testing a *recursive* version of linear search because, when searching
a long list, a straightforward recursive linear search uses more memory than is
available on most computers.

For completeness, here is a recursive linear search:

In [17]:
def contains_recursive_linear(x, lst):
    if lst == []:
        return False
    elif lst[0] == x:
        return True
    else:
        return contains_recursive_linear(x, lst[1:])

These functions should all work identically, so lets write a test function to
check for bugs:

In [22]:
def test_contains(contains):
    """Checks that the passed-in contains function works correctly.
    """
    print(f'calling test_contains({contains.__name__}) ... ')
    table = [
        (3, [], False), (3, [3], True), (1, [3], False), (4, [3], False), 
        (3, [1, 3, 5], True), (1, [1, 3, 5], True), (5, [1, 3, 5], True), 
        (0, [1, 3, 5], False), (2, [1, 3, 5], False),
        (-1, [-1, 3, 51, 100], True), (100, [-1, 3, 51, 100], True), 
        (2, [-1, 3, 51, 100], False), (200, [-1, 3, 51, 100], False)
    ]
    passed = 0
    for x, lst, expected in table:
        actual = contains(x, lst)
        if actual == expected:
            passed += 1
        else:
            print(f'Test {passed+1} failed:')
            print(f'   contains({x}, {lst})')
            print(f'                return: {actual}')
            print(f'              expected: {expected}')

    print(f'... test_contains({contains.__name__}) done: all tests passed')

test_contains(contains_in)
test_contains(contains_index)
test_contains(contains_for_each)
test_contains(contains_for_range)
test_contains(contains_while)
test_contains(contains_sentinel)

calling test_contains(contains_in) ... 
... test_contains(contains_in) done: all tests passed
calling test_contains(contains_index) ... 
... test_contains(contains_index) done: all tests passed
calling test_contains(contains_for_each) ... 
... test_contains(contains_for_each) done: all tests passed
calling test_contains(contains_for_range) ... 
... test_contains(contains_for_range) done: all tests passed
calling test_contains(contains_while) ... 
... test_contains(contains_while) done: all tests passed
calling test_contains(contains_sentinel) ... 
... test_contains(contains_sentinel) done: all tests passed


## The Performance of Linear Search

Recall that when searching a list $n$ elements, **linear search** does $n$
comparisons in the *worst case*, and about $n/2$ comparisons on average. So the
running time of linear search is proportional to $n$, the length of the list.

While this tells us the expected performance of the basic algorithm, it doesn't
help us determine which implementation is fastest. To do that, we'll measure the
performance of each implementation.

## Performance Testing

Which implementation of linear search is fastest? Here's a test function that
times the performance of function by running it on the same test data:

In [24]:
import time

def test_performance():
    contains_functions = [contains_in,
                          contains_index,
                          contains_for_each, 
                          contains_for_range, 
                          contains_while,
                          contains_sentinel,
                          ]

    lst = list(range(1000000))
    print()
    for contains in contains_functions:
        print(f'testing:  {contains.__name__:25}  ', end='')
        start = time.time()
        for i in range(10):
            contains(-1, lst)
        elapsed_time = time.time() - start
        print(f'{elapsed_time:.2f} seconds')

test_performance()


testing:  contains_in                0.06 seconds
testing:  contains_index             0.06 seconds
testing:  contains_for_each          0.16 seconds
testing:  contains_for_range         0.29 seconds
testing:  contains_while             0.75 seconds
testing:  contains_sentinel          0.36 seconds


Some things to note:

- In Python, functions are values, and so you can store them in a list as is
  done here. This makes it easy to loop through the list and test each function
  individually.

- The function `contains_recursive_linear` is commented out because it gets
  stopped by Python after about 1000 recursive calls, and so it can't run on a
  large list like `lst`.

- We always search for the value -1, which is guaranteed to *not* be on the
  list. This forces the functions to do the same amount of work.

- The function `time.time()` returns the current time in seconds. Calling it
  before and after running the tests lets us calculate the total elapsed time.

Here's a typical output on my computer:

```
testing:  contains_in                0.06 seconds
testing:  contains_index             0.06 seconds
testing:  contains_for_each          0.16 seconds
testing:  contains_for_range         0.29 seconds
testing:  contains_while             0.75 seconds
testing:  contains_sentinel          0.36 seconds
```

The built-in `in` and `index` functions are usually the fastest, followed by the
for-each version.